# Example usage of LlammaIndex for RAG


In [1]:
# Set up to use local modules
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # Add parent directory to path
sys.path.insert(0, module_path)

## Gemini with LlammaIndex


In [30]:
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage

from src import utils

import pyprojroot

import dotenv

dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# TODO continue here

In [3]:
# Example with complete
resp = Gemini().complete("Write a poem about a magic backpack")
print(resp)

**The Magic Backpack**

In a realm of wonder, where dreams take flight,
There lies a backpack, with powers of light.
Its fabric gleams, a vibrant hue,
A portal to realms both old and new.

With every step, it whispers secrets untold,
Of adventures waiting, stories to be unrolled.
Its compartments hold treasures beyond compare,
A world of magic, beyond compare.

A book of spells, with pages that ignite,
Unveiling secrets, casting spells of light.
A compass that guides, through paths unknown,
Leading to wonders, where dreams have grown.

A telescope that pierces the veil of night,
Revealing stars that dance in celestial flight.
A wand that grants wishes, with a gentle sway,
Transforming dreams into a vibrant display.

But beware, young traveler, for power can deceive,
The backpack's magic, you must wisely conceive.
For with great power, comes great responsibility,
Use it for good, with utmost humility.

So don the backpack, and embark on your quest,
Where wonders await, and dreams are bl

In [5]:
# Example with chat
messages = [
    ChatMessage(role="user", content="Hello friend!"),
    ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    ChatMessage(role="user", content="Help me decide what to have for dinner."),
]
resp = Gemini().chat(messages)
print(resp)

assistant: Ahoy there, matey! Let's set sail on a culinary adventure and find the perfect dinner for ye. What be yer cravings this fine evening? Arr, consider these options:

* **Grilled Salmon with Roasted Veggies:** Avast! A hearty and healthy choice that'll keep ye satisfied. Grill up some succulent salmon and pair it with roasted veggies like carrots, potatoes, and asparagus.

* **Shepherd's Pie:** Shiver me timbers! A classic comfort food that'll warm ye to the bones. Layer ground beef, veggies, and a creamy mashed potato topping in a baking dish.

* **Chicken Tikka Masala:** Ahoy, landlubber! Embark on a flavorful journey with this Indian dish. Tender chicken marinated in yogurt and spices, simmered in a creamy tomato sauce.

* **Spaghetti and Meatballs:** Weigh anchor and set course for Italy! Savor the classic combination of tender spaghetti, savory meatballs, and a rich tomato sauce.

* **Tacos:** Avast! Ahoy! Ahoy! Set sail for Mexico with these customizable delights. Fill to

In [14]:
# Example with streaming completion
llm = Gemini()
resp = llm.stream_complete(
    "The story of Sourcrust, the bread creature, is really interesting. It all started when..."
)

In [15]:
for r in resp:
    print(r.text, end="")

In the quaint bakery of "Knead to Know," amidst the aroma of freshly baked bread, a peculiar tale unfolded. It began with a humble loaf of sourdough, left to rise overnight in a warm corner.

As the yeast danced and multiplied, a strange transformation occurred. The dough began to quiver, its surface bubbling and contorting. To the astonishment of the baker, a tiny creature emerged from the depths of the loaf.

With a crusty exterior and a soft, doughy interior, the creature came to be known as Sourcrust. Its eyes sparkled with a mischievous glint, and its tiny hands kneaded the air as if it were shaping an invisible loaf.

From that day forward, Sourcrust became an enigmatic presence in the bakery. It would leap from shelf to shelf, leaving a trail of breadcrumbs in its wake. It had a voracious appetite for flour and water, and it possessed an uncanny ability to communicate with the other baked goods.

The croissants whispered secrets to Sourcrust, the baguettes shared their tales of 

In [22]:
# Example with streaming chat
llm = Gemini(model="models/gemini-pro")
messages = [
    ChatMessage(role="user", content="Hello friend!"),
    ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    ChatMessage(role="user", content="Help me decide what to have for dinner."),
]
resp = llm.stream_chat(messages)

In [23]:
for r in resp:
    print(r.delta, end="")

Ahoy there, matey! Let's set sail on a culinary adventure and find the perfect dinner for ye. What be yer cravings, landlubber?

## WIP: RAG with llm=Gemini


In [53]:
from pypdf import PdfReader
from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_inde
import faiss

# From https://www.llamaindex.ai/blog/how-i-built-the-streamlit-llm-hackathon-winning-app-finsight-using-llamaindex-9dcf6c46d7a0


def process_pdf(pdf):
    file = PdfReader(pdf)
    text = ""
    for page in file.pages:
        text += str(page.extract_text())

    doc = Document(text=text)
    return [doc]


def get_vector_index(documents):
    llm = Gemini(model="models/gemini-pro")
    faiss_index = faiss.IndexFlatL2(d)
    vector_store = FaissVectorStore(faiss_index=faiss_index)

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    service_context = ServiceContext.from_defaults(llm=llm)

    index = VectorStoreIndex.from_documents(
        documents, service_context=service_context, storage_context=storage_context
    )

    return index

In [54]:
pdf_path = pyprojroot.here("data/vaswani_et_al_2017.pdf")
# pdf_file = open(pdf_path, "rb")
documents = process_pdf(pdf_path)
index = get_vector_index(documents)
engine = index.as_query_engine()
query = "How has Microsoft performed in this fiscal year?"
response = engine(query)

NameError: name 'd' is not defined

Alternative attempt
https://blog.streamlit.io/build-a-chatbot-with-custom-data-sources-powered-by-llamaindex/


In [58]:
import streamlit as st
import google.generativeai as genai


from llama_index.core import (
    VectorStoreIndex,
    ServiceContext,
    Document,
    SimpleDirectoryReader,
)


# @st.cache_resource(show_spinner=False)
def load_data():
    llm = Gemini(model="models/gemini-pro")
    # pdf_path = pyprojroot.here("data/vaswani_et_al_2017.pdf")
    pdf_path = pyprojroot.here("data")
    reader = SimpleDirectoryReader(input_dir=pdf_path, recursive=True)
    docs = reader.load_data()
    service_context = ServiceContext.from_defaults(llm=llm)
    index = VectorStoreIndex.from_documents(docs, service_context=service_context)
    return index


index = load_data()

/tmp/ipykernel_12615/2461464375.py:20: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******